In [ ]:
pip install pyobjc

In [7]:
import time
import urllib.request
import json
import requests
import os
import pygetwindow as gw
 
item = []

room_id = input('Please enter the room id:')
 
 
# Create and open a file at the beginning of the program
file_path = f'{room_id}房间的弹幕.txt'
with open(file_path, 'a', encoding='utf-8') as fp:
    pass  # Create an empty file
 
os.startfile(file_path)
 
 
def get_mes():
    # Process request to get Bilibili live comments
    url = f'https://api.live.bilibili.com/xlive/web-room/v1/dM/gethistory?roomid={room_id}&room_type=0'
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36 Edg/117.0.2045.43'
    }
    request = urllib.request.Request(url=url, headers=headers)
 
    response = json.loads(requests.get(url=url, headers=headers).text)
 
    return response
 
 
def deal_mes(response):
    # Process bullet comment data
    response = response['data']['room']
    count = 1
 
    for i in response:
        name = i['nickname']
        text = i['text']
        timeline = i['timeline']
 
        if text not in item:
            with open(file_path, 'a', encoding='utf-8') as fp:
                fp.write(f'{timeline},{name}: {text}\n')
                window = gw.getWindowsWithTitle(f'{room_id}房间的弹幕')
                if window:
                    # Close the window, assuming we want to close the first matched window
                    window[0].close()
                    os.startfile(file_path)
                else:
                    print(f'已获取捕捉到弹幕,第{count}条，实时捕捉弹幕中!')
                    count = count+1
 
 
            item.append(text)
        else:
            print('还没有新的弹幕哦，快点发送弹幕哦！ps.由于b站设置，可能需要2分钟，新的弹幕才会被爬取到！')
            time.sleep(5)
 
 
while True:
    response = get_mes()
    deal_mes(response)

AttributeError: module 'os' has no attribute 'startfile'

In [ ]:
import requests
import os
import time

room_id = input('Please enter the room id:')
file_path = f'{room_id}房间的弹幕.txt'
with open(file_path, 'a', encoding='utf-8') as fp:
    pass

def get_mes():
    url = f'https://api.live.bilibili.com/xlive/web-room/v1/dM/gethistory?roomid={room_id}&room_type=0'
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'}
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Will raise HTTPError for bad requests
        return response.json()
    except requests.RequestException as e:
        print(f"Error fetching data: {e}")
        return None

def deal_mes(response):
    if response and 'data' in response and 'room' in response['data']:
        room_data = response['data']['room']
        for i in room_data:
            name = i['nickname']
            text = i['text']
            timeline = i['timeline']
            with open(file_path, 'a', encoding='utf-8') as fp:
                fp.write(f'{timeline},{name}: {text}\n')

while True:
    response = get_mes()
    if response:
        deal_mes(response)
    time.sleep(5)  # Wait for 5 seconds before next API call
